In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/weather-dataset-rattle-package/weatherAUS.csv")

In [ ]:
df.head()

In [ ]:
cols = list(df.columns)
cols

In [ ]:
df.describe().T


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

X = df.drop('RainTomorrow', axis=1)
X.head()

In [ ]:
y = df[["RainTomorrow"]]
y.head()

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(X.corr())
plt.show()

In [ ]:
cor_matrix = df.corr().abs()
print(cor_matrix)

In [ ]:
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))
print(upper_tri)

In [ ]:
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]
print(); print(to_drop)

In [ ]:
df = df.drop(['Pressure3pm', 'Temp3pm', 'Date', 'Location'], axis=1)
print(df.head())

In [ ]:
ohe = pd.get_dummies(data=df, columns=['WindGustDir','WindDir9am','WindDir3pm'])
ohe.info()

In [ ]:
from sklearn import preprocessing
from numpy import array

ohe['RainToday'] = df['RainToday'].astype(str)
ohe['RainTomorrow'] = df['RainTomorrow'].astype(str)

lb = preprocessing.LabelBinarizer()

ohe['RainToday'] = lb.fit_transform(ohe['RainToday'])
ohe['RainTomorrow'] = lb.fit_transform(ohe['RainTomorrow'])

In [ ]:
y = ohe['RainTomorrow']
X = ohe.drop(['RainTomorrow'], axis=1)

In [ ]:
# sns.pairplot(X)
# plt.show()

X.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit_transform(X_train)

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier

model = make_pipeline(
    SimpleImputer(),
    RandomForestClassifier(n_jobs=4, verbose=True, n_estimators=200, max_depth=10, criterion='gini', )
)

model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

In [ ]:
model.score(X_train, y_train)

In [ ]:
from sklearn.metrics import f1_score

pred = model.predict(X_test)
print(f1_score(y_test, pred))